### Este notebook contém os comandos para estruturar o esquema gold.
1. Criar e popular a tabela *total_pago_por_municipio*.

* A tabela *total_pago_por_municipio* é criada e populada a partir das tabelas:
  * *bpc.silver.municipios_pagador*
  * *bpc.silver.municipios_ibge*
  * *bpc.silver.unidade_federacao*
  * *bpc.silver.regiao*

In [0]:
%sql

-- 1. Cria e popula a tabela 'total_pago_por_municipio'
CREATE OR REPLACE TABLE bpc.gold.total_pago_por_municipio
USING delta
COMMENT 'A tabela contém dados sobre o total de pagamentos realizados no âmbito do programa BPC por município no Brasil. Inclui informações como o código do município, o valor total pago, o nome do município, o nome do estado e o nome da região. Esses dados podem ser utilizados para análises regionais dos pagamentos do BPC, para compreender a distribuição financeira entre os municípios e para identificar tendências nos pagamentos de assistência social.'
AS
SELECT
  CAST(mp.Cod_Mun AS INT) AS Cod_Mun,
  CAST(SUM(mp.bpc_val) AS DECIMAL(18,2)) AS bpc_val,
  CAST(mi.NM_MUN AS STRING)AS NM_MUN,
  CAST(uf.NM_UF AS STRING) AS NM_UF,
  CAST(rg.NM_REGIAO AS STRING) AS NM_REGIAO
FROM
  bpc.silver.municipios_pagador mp
    JOIN bpc.silver.municipios_ibge mi
      ON mp.Cod_Mun = mi.CD_MUN
    JOIN bpc.silver.unidade_federacao uf
      ON mi.CD_UF = uf.CD_UF
    JOIN bpc.silver.regiao rg
      ON uf.CD_REGIAO = rg.CD_REGIAO
WHERE
  mp.Cod_Mun IS NOT NULL
  AND mp.bpc_val IS NOT NULL
  AND mi.NM_MUN IS NOT NULL
  AND uf.NM_UF IS NOT NULL
  AND rg.NM_REGIAO IS NOT NULL
GROUP BY
  mp.Cod_Mun,
  mi.NM_MUN,
  uf.NM_UF,
  rg.NM_REGIAO
ORDER BY
  bpc_val DESC;

-- 2. Adiciona comentários nas colunas após a criação da tabela.
-- Nota: O CREATE TABLE AS SELECT não permite definir comentários de coluna diretamente na instrução.
ALTER TABLE bpc.gold.total_pago_por_municipio CHANGE COLUMN Cod_Mun Cod_Mun INT
    COMMENT 'Código IBGE do município ao qual o dado se refere.';

ALTER TABLE bpc.gold.total_pago_por_municipio CHANGE COLUMN bpc_val bpc_val DECIMAL(18,2)
    COMMENT '**Valor total** pago de BPC para todas as categorias por município.';

ALTER TABLE bpc.gold.total_pago_por_municipio CHANGE COLUMN NM_MUN NM_MUN STRING
    COMMENT 'Nome do município.';

ALTER TABLE bpc.gold.total_pago_por_municipio CHANGE COLUMN NM_UF NM_UF STRING
    COMMENT 'Nome da unidade federativa do Brasil.';

ALTER TABLE bpc.gold.total_pago_por_municipio CHANGE COLUMN NM_REGIAO NM_REGIAO STRING
    COMMENT 'Nome da região do Brasil.';

-- 3. Visualiza as 3 primeiras linhas da tabela 'total_pago_por_municipio' recém criada.
SELECT *
FROM bpc.gold.total_pago_por_municipio
LIMIT 3;

Cod_Mun,bpc_val,NM_MUN,NM_UF,NM_REGIAO
110020,344950310.39,Porto Velho,Rondônia,Norte
140010,306640941.63,Boa Vista,Roraima,Norte
120040,305025148.03,Rio Branco,Acre,Norte


2. Tabela *total_pago_por_municipio* criada.

![Municipios pagador](imagens/05-Esquema_gold-Tabela_total_pago_por_municipio.png)

3. Criar e popular a tabela *total_pago_por_unidade_federacao*.

* A tabela *total_pago_por_unidade_federacao* é criada e populada a partir da tabela *total_pago_por_municipio*.

In [0]:
%sql

-- 1. Cria e popula a tabela 'total_pago_por_unidade_federacao'
CREATE OR REPLACE TABLE bpc.gold.total_pago_por_unidade_federacao
USING delta
COMMENT 'A tabela contém dados sobre o total de pagamentos realizados no âmbito do programa BPC por unidade federativa no Brasil. Inclui informações como o nome da unidade federativa, o valor total pago e o nome da região. Esses dados podem ser utilizados para análises regionais dos pagamentos do BPC para compreender a distribuição financeira entre as unidades federativas e para identificar tendências nos pagamentos de assistência social.'
AS
SELECT
  CAST(NM_UF AS STRING) AS NM_UF,
  CAST(SUM(bpc_val) AS DECIMAL(18,2)) AS bpc_val,
  CAST(NM_REGIAO AS STRING) AS NM_REGIAO
FROM
  bpc.gold.total_pago_por_municipio
GROUP BY
  NM_UF,
  NM_REGIAO
ORDER BY
  bpc_val DESC;

-- 2. Adiciona comentários nas colunas após a criação da tabela.
-- Nota: O CREATE TABLE AS SELECT não permite definir comentários de coluna diretamente na instrução.
ALTER TABLE bpc.gold.total_pago_por_unidade_federacao CHANGE COLUMN NM_UF NM_UF STRING
    COMMENT 'Nome da unidade federativa do Brasil.';

ALTER TABLE bpc.gold.total_pago_por_unidade_federacao CHANGE COLUMN bpc_val bpc_val DECIMAL(18,2)
    COMMENT '**Valor total** pago de BPC para todas as categorias por unidade federativa.';

ALTER TABLE bpc.gold.total_pago_por_unidade_federacao CHANGE COLUMN NM_REGIAO NM_REGIAO STRING
    COMMENT 'Nome da região do Brasil.';

-- 3. Visualiza as 3 primeiras linhas da tabela 'total_pago_por_unidade_federacao' recém criada.
SELECT *
FROM bpc.gold.total_pago_por_unidade_federacao
LIMIT 3;

NM_UF,bpc_val,NM_REGIAO
Rio Grande do Sul,1495154813.64,Sul
Paraná,1085480771.83,Sul
Mato Grosso do Sul,792413757.76,Centro-oeste


4. Tabela *total_pago_por_unidade_federacao* criada.

![Municipios pagador](imagens/05-Esquema_gold-Tabela_total_pago_por_unidade_federacao.png)

5. Criar e popular a tabela *total_pago_por_regiao*.

* A tabela *total_pago_por_regiao* é criada e populada a partir da tabela *total_pago_por_unidade_federacao*.

In [0]:
%sql

-- 1. Cria e popula a tabela 'total_pago_por_regiao'
CREATE OR REPLACE TABLE bpc.gold.total_pago_por_regiao
USING delta
COMMENT 'A tabela contém dados sobre o total de pagamentos realizados no âmbito do programa BPC por região do Brasil. Inclui informações como o nome da região e o valor total pago. Esses dados podem ser utilizados para análises regionais dos pagamentos do BPC para compreender a distribuição financeira entre as regiões e para identificar tendências nos pagamentos de assistência social.'
AS
SELECT
  CAST(NM_REGIAO AS STRING) AS NM_REGIAO,
  CAST(SUM(bpc_val) AS DECIMAL(18,2)) AS bpc_val
FROM
  bpc.gold.total_pago_por_unidade_federacao
GROUP BY
  NM_REGIAO
ORDER BY
  bpc_val DESC;

-- 2. Adiciona comentários nas colunas após a criação da tabela.
ALTER TABLE bpc.gold.total_pago_por_regiao CHANGE COLUMN NM_REGIAO NM_REGIAO STRING
    COMMENT 'Nome da região do Brasil.';

ALTER TABLE bpc.gold.total_pago_por_regiao CHANGE COLUMN bpc_val bpc_val DECIMAL(18,2)
    COMMENT '**Valor total** pago de BPC para todas as categorias por região.';

-- 3. Visualiza a primeira linha da tabela 'total_pago_por_regiao' recém criada.
SELECT *
FROM bpc.gold.total_pago_por_regiao
LIMIT 1;

NM_REGIAO,bpc_val
Sul,2782006691.32


6. Tabela *total_pago_por_regiao* criada.

![Municipios pagador](imagens/05-Esquema_gold-Tabela_total_pago_por_regiao.png)

7. Esquema **gold** criado.